<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3e577f49d17f983a59d2155bb8dec5884505a15db8754cebeb65da93426d3c77
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-28 09:30:31
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -34.49 K (-0.24%)
Current PnL: -22.72 L (-14.89%)
CY Booked + Current PnL: -8.83 L (-5.79%)
-------------------
Total profit:  1.61 L
Total loss:  -24.33 L
-------------------
Total Booked + Current PnL: 18.15 L (14.45%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.71%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.13 L (61.6%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,-0.67,-14.69,17.33,0.09,14898.0,-14808.0,85969.0,99.57,46.0,M-SC,3.60,253.0,-0.99,0.60,12.94,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-0.59,-9.55,10.57,0.02,16690.0,-16664.0,157895.0,140.51,65.0,M-SC,10.40,234.0,-1.00,1.10,37.34,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,-0.98,8.29,10.53,19.69,21865.0,15903.0,207645.0,-2.32,52.0,X-MC,3.33,78.0,0.73,1.45,15.66,XY25,NTT,AC
43,ITC,452.0,0.50,-1.22,11.80,10.44,23329.0,-2435.0,197703.0,-41.15,46.0,X-LC,1.00,5.0,-0.10,1.38,4.28,X40,NTT,FMCG
52,MEDANTA,1486.0,-0.79,3.55,19.83,24.08,26067.0,4501.0,131451.0,-4.85,46.0,X-SC,5.89,89.0,0.17,0.92,23.86,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,2.49,-3.25,40.23,35.67,122931.0,-10272.0,305570.0,-14.67,61.0,X-LC,5.73,4.0,-0.08,2.14,10.15,X40N,ATH,FMCG
27,GLAXO,3466.20,1.22,1.15,37.57,39.15,75729.0,2284.0,201568.0,-18.74,41.0,X-MC,10.02,60.0,0.03,1.41,30.48,X40,ATH,PHARMA
38,INDIAMART,4810.62,1.15,0.98,104.72,106.72,130417.0,1203.0,124539.0,-50.06,42.0,H-SC,2.78,139.0,0.01,0.87,24.58,AR,ATH,MISC
81,VALIANTORG,838.00,1.02,-46.42,225.06,74.15,197270.0,-75953.0,87652.0,-257.83,29.0,H-SC,17.48,149.0,-0.39,0.61,12.36,XR,NTT,CHEMICALS
18,COFFEEDAY,80.00,0.91,-38.05,118.34,35.27,83251.0,-43200.0,70349.0,-51.71,38.0,L-SC,16.32,270.0,-0.52,0.49,72.18,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,KANSAINER,340.00,-1.36,-22.07,45.60,13.47,95833.0,-59508.0,210159.0,-67.92,35.0,H-SC,1.63,158.0,-0.62,1.47,6.86,XY24,NTT,PAINTS
69,STARHEALTH,761.00,-1.34,-6.26,57.10,47.26,150466.0,-17609.0,263514.0,27.90,45.0,H-SC,15.38,171.0,-0.12,1.84,41.18,XY24,NTT,INSURANCE
0,5PAISA,593.00,-1.29,-35.95,76.28,12.90,106748.0,-78554.0,139942.0,106.73,55.0,H-SC,13.10,161.0,-0.74,0.98,16.36,OX40N,NTT,FINANCE
26,GILLETTE,11206.78,-1.27,2.22,29.54,32.42,76670.0,5649.0,259545.0,-15.97,48.0,X-SC,7.73,85.0,0.07,1.81,17.66,X40,ATH,FMCG
48,KPIGREEN,730.66,-1.14,-12.74,68.41,46.95,74793.0,-15967.0,109330.0,-19.22,28.0,H-SC,2.29,154.0,-0.21,0.76,36.96,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,1.15,0.98,104.72,106.72,130417.0,1203.0,124539.0,-50.06,42.0,H-SC,2.78,139.0,0.01,0.87,24.58,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.32,-0.85,36.79,35.63,76448.0,-1779.0,207795.0,-14.97,43.0,H-MC,3.77,119.0,-0.02,1.45,15.68,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.87,-5.38,119.72,107.91,167258.0,-7936.0,139708.0,-22.98,34.0,M-SC,10.69,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.59,-9.55,10.57,0.02,16690.0,-16664.0,157895.0,140.51,65.0,M-SC,10.40,234.0,-1.00,1.10,37.34,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.67,-14.69,17.33,0.09,14898.0,-14808.0,85969.0,99.57,46.0,M-SC,3.60,253.0,-0.99,0.60,12.94,OX40N,NTT,DURABLES
67,SIS,528.00,-0.97,-21.59,54.66,21.26,47585.0,-23975.0,87057.0,2056.99,57.0,H-SC,3.71,166.0,-0.50,0.61,17.62,OX40N,NTT,MISC
47,KANSAINER,340.00,-1.36,-22.07,45.60,13.47,95833.0,-59508.0,210159.0,-67.92,35.0,H-SC,1.63,158.0,-0.62,1.47,6.86,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.11,2.47,68.11,72.27,118266.0,4191.0,173639.0,-11.98,59.0,M-LC,3.68,99.0,0.04,1.21,8.80,XR,NTT,IT
38,INDIAMART,4810.62,1.15,0.98,104.72,106.72,130417.0,1203.0,124539.0,-50.06,42.0,H-SC,2.78,139.0,0.01,0.87,24.58,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.11,2.47,68.11,72.27,118266.0,4191.0,173639.0,-11.98,59.0,M-LC,3.68,99.0,0.04,1.21,8.80,XR,NTT,IT
38,INDIAMART,4810.62,1.15,0.98,104.72,106.72,130417.0,1203.0,124539.0,-50.06,42.0,H-SC,2.78,139.0,0.01,0.87,24.58,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.32,-0.85,36.79,35.63,76448.0,-1779.0,207795.0,-14.97,43.0,H-MC,3.77,119.0,-0.02,1.45,15.68,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.87,-5.38,119.72,107.91,167258.0,-7936.0,139708.0,-22.98,34.0,M-SC,10.69,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
1,ABB,7934.00,-0.10,-3.95,51.56,45.57,129560.0,-10339.0,251280.0,-38.27,59.0,H-MC,5.72,120.0,-0.08,1.76,6.92,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.23,-35.64,110.06,35.20,91148.0,-45853.0,82817.0,8.59,24.0,X-SC,13.36,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.11,-8.98,26.60,15.23,48505.0,-17996.0,182351.0,-22.11,29.0,X-MC,6.97,66.0,-0.37,1.27,20.37,X40N,NTT,REALTY
76,TMPV,600.00,0.28,-14.98,67.22,42.18,157976.0,-41396.0,235014.0,-24.30,29.0,X-LC,1.92,3.0,-0.26,1.64,1.80,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.37,-14.15,72.92,48.45,125520.0,-28372.0,172134.0,-20.30,30.0,X-MC,7.54,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
55,PGHH,17907.65,0.23,-4.32,39.81,33.76,76489.0,-8685.0,192135.0,-32.51,35.0,X-MC,3.87,57.0,-0.11,1.34,0.65,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.50,-1.22,11.80,10.44,23329.0,-2435.0,197703.0,-41.15,46.0,X-LC,1.00,5.0,-0.10,1.38,4.28,X40,NTT,FMCG
3,ACC,3906.0,-0.38,-21.70,109.65,64.15,204289.0,-51641.0,186310.0,-54.13,51.0,X-SC,1.54,82.0,-0.25,1.30,4.51,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.52,-11.47,41.78,25.51,68833.0,-21345.0,164750.0,1.14,63.0,H-LC,1.87,49.0,-0.31,1.15,21.41,AR,ATH,ELECTRICAL
15,CAMS,4762.0,-0.42,1.63,22.79,24.79,58334.0,4105.0,255961.0,-3.02,48.0,X-SC,1.87,86.0,0.07,1.79,25.15,X40N,NTT,MISC
76,TMPV,600.0,0.28,-14.98,67.22,42.18,157976.0,-41396.0,235014.0,-24.30,29.0,X-LC,1.92,3.0,-0.26,1.64,1.80,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.23,-35.64,110.06,35.20,91148.0,-45853.0,82817.0,8.59,24.0,X-SC,13.36,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.37,-14.15,72.92,48.45,125520.0,-28372.0,172134.0,-20.30,30.0,X-MC,7.54,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.95,-5.80,33.83,26.07,52179.0,-9500.0,154240.0,-30.68,36.0,X-MC,9.55,55.0,-0.18,1.08,0.06,X40,ATH,APPARELS
55,PGHH,17907.65,0.23,-4.32,39.81,33.76,76489.0,-8685.0,192135.0,-32.51,35.0,X-MC,3.87,57.0,-0.11,1.34,0.65,X40,ATH,FMCG
56,QUESS,424.00,-0.35,-28.26,99.13,42.84,46226.0,-18374.0,46632.0,-53.17,40.0,X-SC,37.60,83.0,-0.40,0.33,1.47,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.35,-28.26,99.13,42.84,46226.0,-18374.0,46632.0,-53.17,40.0,X-SC,37.60,83.0,-0.40,0.33,1.47,XY24,NTT,MISC
60,RELAXO,1176.00,-0.52,-47.16,188.24,52.31,144388.0,-68456.0,76704.0,-43.76,37.0,X-SC,4.54,91.0,-0.47,0.54,2.67,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.23,-35.64,110.06,35.20,91148.0,-45853.0,82817.0,8.59,24.0,X-SC,13.36,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.72,-6.64,30.23,21.58,39551.0,-9306.0,130833.0,-52.96,35.0,X-MC,6.43,56.0,-0.24,0.91,15.83,X40N,ATH,IT
52,MEDANTA,1486.00,-0.79,3.55,19.83,24.08,26067.0,4501.0,131451.0,-4.85,46.0,X-SC,5.89,89.0,0.17,0.92,23.86,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.06,-14.83,40.05,19.28,118008.0,-51304.0,294652.0,-25.89,58.0,X-LC,7.11,1.0,-0.43,2.06,8.93,X40,ATH,IT
41,INFY,2275.00,-0.29,6.32,45.67,54.87,151927.0,19775.0,332663.0,-17.09,61.0,X-LC,2.48,2.0,0.13,2.33,15.13,X40,BTT,IT
76,TMPV,600.00,0.28,-14.98,67.22,42.18,157976.0,-41396.0,235014.0,-24.30,29.0,X-LC,1.92,3.0,-0.26,1.64,1.80,XY24,NTT,AUTO
82,VBL,671.64,2.49,-3.25,40.23,35.67,122931.0,-10272.0,305570.0,-14.67,61.0,X-LC,5.73,4.0,-0.08,2.14,10.15,X40N,ATH,FMCG
43,ITC,452.00,0.50,-1.22,11.80,10.44,23329.0,-2435.0,197703.0,-41.15,46.0,X-LC,1.00,5.0,-0.10,1.38,4.28,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.13,-36.78,110.21,32.89,53792.0,-28398.0,48809.0,-702.61,65.0,L-MC,5.75,259.0,-0.53,0.34,34.42,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.27,-12.27,107.07,81.67,88122.0,-11507.0,82303.0,7400.00,53.0,L-SC,19.60,271.0,-0.13,0.58,59.38,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.31,-12.14,58.82,39.54,52280.0,-12277.0,88882.0,-31.30,66.0,M-SC,6.66,220.0,-0.23,0.62,25.33,AR,ATH,AUTO
70,SURYODAY,216.00,-0.24,-20.08,53.96,23.03,77220.0,-35965.0,143106.0,55.70,47.0,H-SC,9.56,167.0,-0.47,1.00,41.99,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.59,-9.55,10.57,0.02,16690.0,-16664.0,157895.0,140.51,65.0,M-SC,10.40,234.0,-1.00,1.10,37.34,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.59,-9.55,10.57,0.02,16690.0,-16664.0,157895.0,140.51,65.0,M-SC,10.40,234.0,-1.00,1.10,37.34,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.36,0.27,20.63,20.95,52078.0,669.0,252437.0,-0.77,73.0,X-LC,15.15,31.0,0.01,1.76,36.00,X40,ATH,PAINTS
50,LTIM,7201.88,0.03,10.19,19.48,31.66,52837.0,25087.0,271238.0,6.01,64.0,H-LC,13.52,43.0,0.47,1.90,50.50,X200,ATH,IT
40,INDUSINDBK,1800.00,-0.13,-36.78,110.21,32.89,53792.0,-28398.0,48809.0,-702.61,65.0,L-MC,5.75,259.0,-0.53,0.34,34.42,XR,NTT,BANKS
32,HCLTECH,1922.01,-0.25,5.45,18.28,24.73,46634.0,13185.0,255109.0,10.22,62.0,X-LC,4.78,7.0,0.28,1.78,21.52,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,44.78
2,50,75.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.41
MC    29.18
LC    25.36
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.77
X40      21.21
X40N     11.97
XR        9.50
XY25      9.12
AR        9.12
OX40N     7.62
X200      1.90
SR        0.98
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.85
X-MC    22.69
X-LC    20.12
M-SC    11.89
X-SC     8.26
H-MC     4.77
H-LC     3.05
L-SC     1.41
M-MC     1.38
M-LC     1.21
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.88,-4.18,37.87
IT,13.80,-14.55,73.69
FINANCE,9.45,-14.60,63.02
MISC,7.41,-22.47,72.64
ELECTRICAL,6.00,-9.37,49.44
PAINTS,5.93,-8.63,24.76
INSURANCE,4.40,-0.44,35.84
PHARMA,3.88,-0.49,32.65
AUTO,3.38,-18.79,69.72


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3060818.0,21
XR,1299978.0,13
AR,1267709.0,10
X40,1002747.0,14
X40N,734952.0,9
OX40N,702463.0,10
XY25,336145.0,6
SR,280660.0,2
MH,74793.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3565309.0,25
M-SC,1355878.0,15
X-MC,1233424.0,16
X-LC,883601.0,11
X-SC,738874.0,8
H-MC,395958.0,3
L-SC,259462.0,3
H-LC,121670.0,2
M-LC,118266.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1220405.0      6
           AR         871595.0      5
           XR         796417.0      7
M-SC       XY24       781540.0      6
X-MC       X40        442953.0      7
X-LC       X40        391976.0      5
X-MC       XY24       364789.0      3
H-SC       OX40N      321439.0      4
X-SC       X40N       294474.0      3
M-SC       OX40N      292935.0      5
H-SC       SR         280660.0      2
X-SC       XY24       276582.0      3
X-MC       X40N       248324.0      4
X-LC       XY24       227552.0      2
H-MC       AR         206008.0      2
X-LC       X40N       192154.0      2
H-MC       XY24       189950.0      1
X-MC       XY25       177358.0      2
L-SC       XR         171373.0      2
X-SC       X40        167818.0      2
M-SC       XR         160130.0      2
           AR         121273.0      2
M-LC       XR         118266.0      1
L-SC       OX40N       88089.0      1
H-SC       MH          74793.0      1
X-LC       XY25        71919.0      2
H-LC       AR          68833.0      1
L-MC       XR          53792.0      1
H-LC       X200        52837.0      1
M-MC       XY25        47190.0      1
L-LC       XY25        39678.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
